<a href="https://colab.research.google.com/github/SharvariChawade/Web-Scraping-NLP/blob/main/WebScrapping_%2B_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os

In [ ]:
df = pd.read_csv('/content/input.csv')

In [ ]:
i = 37

for url in (df['URL']):
  path = '/content/allscrapedfiles'
  html_text = requests.get(url,headers={"User-Agent": "XY"}).text
  soup = BeautifulSoup(html_text,'lxml')
  article_title = soup.find('h1',class_ = 'entry-title')
  if(article_title == None):
    i+=1
    continue
  else:
    article_texts = soup.find_all('p')
    content = ''
    for article_text in article_texts:
      content = content + article_text.text +'\n'
    temp = f'id{i}.txt'
    path = os.path.join(path,temp)
    f = open(path, "w")
    f.write(f'{article_title.text}\n{content}')
    f.close()
    i+=1

In [ ]:
import nltk
nltk.download('punkt')

import re

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
f = open('/content/negative-words.txt','r',encoding = "ISO-8859-1")
negativewords = f.read()
negativewords = list(negativewords.split('\n'))

f = open('/content/positive-words.txt','r',encoding = "ISO-8859-1")
positivewords = f.read()
positivewords = list(positivewords.split('\n'))

stop_words = []
stopwords = ''
for file in os.listdir('/content'):
  if 'StopWords' in file:
    path = os.path.join('/content',file)
    f = open(path,'r',encoding = "ISO-8859-1")
    stopword = f.read().lower()
    stopwords += stopword

stop_words = list(stopwords.split('\n'))

positive_words = []
negative_words = []
for w in positivewords:
  if w not in stop_words:
    positive_words.append(w)

for w in negativewords:
  if w not in stop_words:
    negative_words.append(w)


In [ ]:
df1 = [['POSITIVE SCORE','NEGATIVE SCORE','POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH']]

In [ ]:
for i in range(37, 151):
  if (i == 44 ):
    df1.append(['NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN'])
    continue
  elif (i == 57 ):
    df1.append(['NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN'])
    continue
  elif (i == 144):
    df1.append(['NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN'])
    continue
  else:
    f = open(f'/content/allscrapedfiles/id{i}.txt','r')
    text = f.read()

    sentences = nltk.sent_tokenize(text)
    text_words = []

    for i in range (len(sentences)):
      review = re.sub('[^a-zA-Z]',' ',sentences[i])
      review = review.lower()
      review = review.split()
      review = [lemmatizer.lemmatize(word) for word in review if not word in stop_words]
      for w in review:
        text_words.append(w)

    pos = 0
    neg = 0

    for w in text_words:
      if w in positive_words:
        pos += 1

    for w in text_words:
      if w in negative_words:
        neg += 1

    polarity = (pos - neg)/(pos+neg)+0.000001
    subjectivity = (pos+neg)/((len(text_words))+0.000001)

    complex_words = []
    for w in text_words:
      if len(w) > 2:
        complex_words.append(w)

    average_sentence_len = len(text_words)/len(sentences)
    percent_complex_words = len(complex_words)/len(text_words)
    fog_index = 0.4*(average_sentence_len+percent_complex_words)
    avg_no_of_word = len(text_words)/len(sentences)
    complex_count = len(complex_words)

    word_count_list = []
    from nltk.corpus import stopwords
    for w in text_words:
      if w not in set(stopwords.words('english')):
        word_count_list.append(w)

    word_count = len(word_count_list)

    syllable = 0
    text = text.replace('\n',' ')
    list_text = text.split(' ')

    for w in list_text:
      for i in range(len(w)):
        if (w[i] == 'a' or w[i] == 'e' or w[i] == 'i' or w[i] == 'o' or w[i] == 'u'):
          if(i != len(w) - 1):
            if(w[i] == 'e' and w[i+1] != 's' and w[i+1] != 'd'):
              syllable += 1

    personal_pronouns = 0

    for i in sentences:
      if('the us' in i):
        i.replace('the us', "THE US")
      l = []
      l = re.findall("I,|we,|my,|ours,|us",i)
      personal_pronouns += len(l)


    chars_in_text = text.replace(' ','')
    avg_word_len = len(chars_in_text)/len(list_text)

    df1.append([pos,neg,polarity,subjectivity,average_sentence_len,percent_complex_words,fog_index,avg_no_of_word,complex_count,word_count,syllable,personal_pronouns,avg_word_len])

In [ ]:
import numpy
 
# create an array
a = numpy.array(df1)
#text = "POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH\n"
# save array into csv file
rows = ["{},{},{},{},{},{},{},{},{},{},{},{},{}".format(a,b,c,d,e,f,g,h,i,j,k,l,m) for a,b,c,d,e,f,g,h,i,j,k,l,m in a]
text = "\n".join(rows)
 
with open('/content/df1.csv', 'w') as f:
    f.write(text)

In [ ]:
df_1 = pd.read_csv('/content/df1.csv')

In [ ]:
output = pd.concat([df, df_1], axis = 1)

In [ ]:
output.to_csv('/content/output.csv')